In [1]:
path = '../input/plant-pathology-2021-fgvc8/'
train_dir = path + 'train_images/'
test_dir = path + 'test_images/'

In [2]:
import pandas as pd
df = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv", dtype=str)
df.labels.value_counts()

scab                               4826
healthy                            4624
frog_eye_leaf_spot                 3181
rust                               1860
complex                            1602
powdery_mildew                     1184
scab frog_eye_leaf_spot             686
scab frog_eye_leaf_spot complex     200
frog_eye_leaf_spot complex          165
rust frog_eye_leaf_spot             120
rust complex                         97
powdery_mildew complex               87
Name: labels, dtype: int64

In [3]:
df['labels'] = df['labels'].astype(str)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen  = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.3)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                 validation_split = 0.3)

train_generator = train_datagen.flow_from_dataframe(dataframe = df,
                                                   directory = train_dir,
                                                   target_size = (150,150),
                                                   x_col = 'image',
                                                   y_col = 'labels',
                                                   batch_size = 256,
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical',
                                                   subset = 'training')

test_generator = test_datagen.flow_from_dataframe(dataframe = df,
                                                 directory = train_dir,
                                                 target_size = (150,150),
                                                 x_col = 'image',
                                                 y_col = 'labels',
                                                 batch_size = 256,
                                                 color_mode = 'rgb',
                                                 class_mode = 'categorical',
                                                 subset = 'validation')

Found 13043 validated image filenames belonging to 12 classes.
Found 5589 validated image filenames belonging to 12 classes.


In [5]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()

model.add(layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation = 'relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D((2,2)))


model.add(layers.Conv2D(256, (3,3), activation = 'relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(12, activation = 'softmax'))

optimizer = optimizers.Adam(lr = 0.001)

model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 256)       2

In [6]:
history = model.fit(train_generator, epochs = 10, validation_data = test_generator)

Epoch 1/10
51/51 [==============================] - 3103s 61s/step - loss: 2.0097 - accuracy: 0.2389 - val_loss: 1.7182 - val_accuracy: 0.3782
Epoch 2/10
51/51 [==============================] - 2959s 58s/step - loss: 1.6870 - accuracy: 0.3698 - val_loss: 1.5446 - val_accuracy: 0.4362
Epoch 3/10
51/51 [==============================] - 2981s 59s/step - loss: 1.5333 - accuracy: 0.4406 - val_loss: 1.3805 - val_accuracy: 0.5076
Epoch 4/10
51/51 [==============================] - 3023s 60s/step - loss: 1.3312 - accuracy: 0.5268 - val_loss: 1.2062 - val_accuracy: 0.5862
Epoch 5/10
51/51 [==============================] - 2844s 56s/step - loss: 1.1393 - accuracy: 0.5989 - val_loss: 1.0821 - val_accuracy: 0.6257
Epoch 6/10
51/51 [==============================] - 2889s 57s/step - loss: 0.9972 - accuracy: 0.6406 - val_loss: 1.0463 - val_accuracy: 0.6429
Epoch 7/10
51/51 [==============================] - 2917s 58s/step - loss: 0.9404 - accuracy: 0.6620 - val_loss: 0.9882 - val_accuracy: 0.6538